### Purpose of notebook
The purpose of this notebook is to pass the YouTube ASL videos through the MoviNet backbone and then save its output back to S3

In [1]:
# Let's not show unnecessary warnings etc
import warnings
warnings.filterwarnings('ignore')

In [2]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [3]:
# AWS credentials and S3 settings
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
s3_URI = 's3://asl-capstone/'

In [4]:
import s3fs
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

import boto3
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')

In [6]:
# Import all required libraries. Keep adding here as you code
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import dask
from dask.distributed import Client, progress, as_completed
tf.keras.backend.clear_session()

In [7]:
# Set working directory
%cd /home/ec2-user/models

/home/ec2-user/models


In [8]:
# Define the model you are going to use
model_version = 'a3'

if model_version=='a3':
  max_frames = 120
  image_dims = (256,256)
elif model_version=='a0':
  max_frames = 50
  image_dims = (172,172)
elif model_version=='a5':
  max_frames = 120
  image_dims = (320,302)

In [9]:
# Get the kinetics-600 action labels
KINETICS_URL = "official/projects/movinet/files/kinetics_600_labels.txt"
with open(KINETICS_URL) as obj:
  labels_600 = [line for line in obj.readlines()]
print("Found %d labels." % len(labels_600))

Found 600 labels.


In [10]:
# Create the backbone using the Movinet model
from official.projects.movinet.modeling import movinet

# Create backbone and model.
backbone = movinet.Movinet(
    model_id=model_version, #change to correspond to model
    causal=False,
    use_external_states=False,
)

2023-11-18 04:25:55.715877: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-18 04:25:55.738908: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-18 04:25:55.740771: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
# Get a list of all the numpy files from S3

paginator = s3.get_paginator('list_objects_v2')
video_files = []
for page in paginator.paginate(Bucket = bucket_name, Prefix = 'youtube-asl/1000-samples/numpy_files/'):
    video_files.extend(content['Key'] for content in page.get('Contents',[]) if content['Key'].endswith(('.npy')))
print(len(video_files))

59042


In [12]:
# Add the S3 prefix
video_files = ['s3://asl-capstone/'+x for x in video_files]

In [13]:
# Let's break this into batches
video_files = video_files[:20000]

In [23]:
# Iteration batch
iteration_1 = video_files[348:5000]
iteration_2 = video_files[5001:10000]
iteration_3 = video_files[10001:15000]
iteration_4 = video_files[15000:]

In [24]:
%%time
iteration_2 = iteration_2[:10]

CPU times: user 36 µs, sys: 1 µs, total: 37 µs
Wall time: 41.5 µs


In [16]:
# %%time
# for vid in tqdm(iteration_2):
#     try:
#         # Generate video embeddings and store to CPU
#         filename = os.path.basename(vid)
#         with fs.open(vid,"rb") as f:
#             vid_file = np.load(f)
#         vid_file = np.expand_dims(vid_file, axis=0)
#         embeddings = backbone(vid_file)[0]['block4_layer9'].cpu().numpy()
#         with fs.open(f"{movinet_uri+filename}","wb") as f:
#             np.save(f,embeddings)
#     except:
#         print(f"Error with processing {vid}")

  0%|                                                                                                                                                                                  | 0/5 [00:00<?, ?it/s]2023-11-18 04:29:22.021732: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x278a4780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-18 04:29:22.021774: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2023-11-18 04:29:22.852413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8903
2023-11-18 04:29:23.651424: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Instructions for updating:
Use tf.identity with explicit device placement instead.


 20%|██████████████████████████████████                                                                                                                                        | 1/5 [00:20<01:22, 20.61s/it]

Error with processing s3://asl-capstone/youtube-asl/1000-samples/numpy_files/4zPLsd-C7cI_19.npy


 40%|████████████████████████████████████████████████████████████████████                                                                                                      | 2/5 [00:41<01:01, 20.61s/it]

...

Error with processing s3://asl-capstone/youtube-asl/1000-samples/numpy_files/4zPLsd-C7cI_20.npy


 80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                  | 4/5 [01:17<00:18, 18.88s/it]

Error with processing s3://asl-capstone/youtube-asl/1000-samples/numpy_files/4zPLsd-C7cI_21.npy


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:36<00:00, 19.23s/it]

Error with processing s3://asl-capstone/youtube-asl/1000-samples/numpy_files/4zPLsd-C7cI_22.npy
CPU times: user 1min 11s, sys: 1.79 s, total: 1min 12s
Wall time: 1min 36s


In [25]:
movinet_uri = 's3://asl-capstone/youtube-asl/1000-samples/movinet/backbone/'

In [26]:
# Start a Dask client
client = Client(processes=False)
# Print the link to the dashboard
print("-------------------------------------------------------------")
print(client.dashboard_link)

-------------------------------------------------------------
http://172.31.13.102:34807/status


In [27]:
def process_video(vid):
  # Add a try except
  try:
      # Generate video embeddings and store to CPU
      filename = os.path.basename(vid)
      with fs.open(vid,"rb") as f:
          vid_file = np.load(f)
      vid_file = np.expand_dims(vid_file, axis=0)
      embeddings = backbone(vid_file)[0]['block4_layer9'].cpu().numpy()
      with fs.open(f"{movinet_uri+filename}","wb") as f:
          np.save(f,embeddings)
  except:
      print(f"Error with processing {vid}")

In [28]:
%%time
# Use Dask's map function to apply the process_video function to each video
futures = client.map(process_video, iteration_2)

# Create a progress bar with tqdm
progress_bar = tqdm(total=len(iteration_2))

# Use as_completed to update the progress bar as futures complete
for future in as_completed(futures):
    result = future.result()  # This blocks until the future is complete
    progress_bar.update()

progress_bar.close()

# Wait for all futures to complete
client.gather(futures)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:17<00:00,  7.73s/it]

CPU times: user 3min 8s, sys: 8.31 s, total: 3min 16s
Wall time: 1min 22s


[None, None, None, None, None, None, None, None, None, None]

In [ ]:
def get_instance_id():
    """Get current instance ID from metadata"""
    url = "http://169.254.169.254/latest/meta-data/instance-id"
    response = requests.get(url)
    return response.text

In [ ]:
def stop_instance(instance_id, region_name='us-west-2'):
    """Stop the EC2 instance"""
    ec2 = boto3.client('ec2', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key, region_name=region_name)
    ec2.stop_instances(InstanceIds=[instance_id])

In [ ]:
# Get the current instance ID
instance_id = get_instance_id()
print(instance_id)
# Stop the instance
stop_instance(instance_id)